In [28]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [29]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle (3).json': b'{"username":"luzyramirez","key":"ea1df442e687202a0846fe0c3a0f47ce"}'}

In [30]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia


udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [34]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


In [35]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)

shape of loaded dataframe (692500, 21)


In [36]:

import matplotlib.pyplot as plt

In [37]:
z.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# Preprocesado de datos
En base a la exploracion de los datos, escogi trabajar con las siguientes columnas:

-ESTU_VALORMATRICULAUNIVERSIDAD

-ESTU_HORASSEMANATRABAJA

-FAMI_ESTRATOVIVIENDA

-FAMI_TIENEINTERNET

RENDIMIENTO_GLOBAL


In [38]:
datos = z[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'RENDIMIENTO_GLOBAL' ]]
datos.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,medio-alto
1,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,bajo
2,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,bajo
3,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,alto
4,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,medio-bajo


Aqui hicimos la limpieza de datos nulos

In [39]:
sum(datos.ESTU_VALORMATRICULAUNIVERSIDAD.isna()), sum(datos.ESTU_HORASSEMANATRABAJA.isna()), sum(datos.FAMI_ESTRATOVIVIENDA.isna()), sum(datos.FAMI_TIENEINTERNET.isna())

(6287, 30857, 32137, 26629)

In [40]:
datos.ESTU_VALORMATRICULAUNIVERSIDAD.values[datos.ESTU_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'
datos.ESTU_HORASSEMANATRABAJA.values[datos.ESTU_HORASSEMANATRABAJA.isna()] = 'no info'
datos.FAMI_ESTRATOVIVIENDA.values[datos.FAMI_ESTRATOVIVIENDA.isna()] = 'no info'
datos.FAMI_TIENEINTERNET.values[datos.FAMI_TIENEINTERNET.isna()] = 'no info'
sum(datos.ESTU_VALORMATRICULAUNIVERSIDAD.isna()), sum(datos.ESTU_HORASSEMANATRABAJA.isna()), sum(datos.FAMI_ESTRATOVIVIENDA.isna()), sum(datos.FAMI_TIENEINTERNET.isna())

(0, 0, 0, 0)

Limpieza de ESTU_VALORMATRICULAUNIVERSIDAD

In [41]:
vmat = {'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Menos de 500 mil': 0.25,
 'Entre 500 mil y menos de 1 millón': 0.75,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Más de 7 millones': 7.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'No pagó matrícula': 0,
 'no info': -1}

datos.ESTU_VALORMATRICULAUNIVERSIDAD = np.r_[[vmat[i] for i in datos.ESTU_VALORMATRICULAUNIVERSIDAD]]
datos.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

<ipython-input-41-ddd73b42eb3e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.ESTU_VALORMATRICULAUNIVERSIDAD = np.r_[[vmat[i] for i in datos.ESTU_VALORMATRICULAUNIVERSIDAD]]


,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,6287


Limpieza de ESTU_HORASSEMANATRABAJA

In [42]:
htra = {'0': 0,
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15,
    'Entre 21 y 30 horas': 25,
    'Más de 30 horas': 35,
    'no info': -1}

datos.ESTU_HORASSEMANATRABAJA = np.r_[[htra[i] for i in datos.ESTU_HORASSEMANATRABAJA]]
datos.ESTU_HORASSEMANATRABAJA.value_counts()

<ipython-input-42-d66c4cb89768>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.ESTU_HORASSEMANATRABAJA = np.r_[[htra[i] for i in datos.ESTU_HORASSEMANATRABAJA]]


,count
ESTU_HORASSEMANATRABAJA,
35,249352
0,116550
15,115857
25,92693
5,87191
-1,30857


Limpieza de FAMI_ESTRATOVIVIENDA

In [43]:
datos.FAMI_ESTRATOVIVIENDA = ['no info' if i in ['Sin Estrato'] else i for i in datos.FAMI_ESTRATOVIVIENDA.values]

<ipython-input-43-570f5984323e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.FAMI_ESTRATOVIVIENDA = ['no info' if i in ['Sin Estrato'] else i for i in datos.FAMI_ESTRATOVIVIENDA.values]


In [44]:
datos.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
no info,35426
Estrato 5,23608
Estrato 6,12605


In [45]:
estra = {'Estrato 1': 1,
 'Estrato 2': 2,
 'Estrato 3': 3,
 'Estrato 4': 4,
 'Estrato 5': 5,
 'Estrato 6': 6,
 'no info': -1}

datos.FAMI_ESTRATOVIVIENDA = np.r_[[estra[i] for i in datos.FAMI_ESTRATOVIVIENDA]]
datos.FAMI_ESTRATOVIVIENDA.value_counts()

<ipython-input-45-20948591d44c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.FAMI_ESTRATOVIVIENDA = np.r_[[estra[i] for i in datos.FAMI_ESTRATOVIVIENDA]]


,count
FAMI_ESTRATOVIVIENDA,
2,232671
3,210685
1,111991
4,65514
-1,35426
5,23608
6,12605


Limpieza de FAMI_TIENEINTERNET

In [46]:
x = datos.FAMI_TIENEINTERNET.values
FAMI_TIENEINTERNET_vals = sorted(np.unique(x))
FAMI_TIENEINTERNET_onehot_vals = {val: np.eye(len(FAMI_TIENEINTERNET_vals))[i] for i,val in enumerate(FAMI_TIENEINTERNET_vals)}
FAMI_TIENEINTERNET_onehot_vals

{'No': array([1., 0., 0.]),
 'Si': array([0., 1., 0.]),
 'no info': array([0., 0., 1.])}

In [47]:
FAMI_TIENEINTERNET_onehot_enc = np.r_[[FAMI_TIENEINTERNET_onehot_vals[i] for i in datos.FAMI_TIENEINTERNET]]
FAMI_TIENEINTERNET_onehot_enc

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [48]:
FAMI_TIENEINTERNET_df = pd.DataFrame(FAMI_TIENEINTERNET_onehot_enc, columns=[f"FAMI_TIENEINTERNET__{v}" for v in FAMI_TIENEINTERNET_onehot_vals])
FAMI_TIENEINTERNET_df

,FAMI_TIENEINTERNET__No,FAMI_TIENEINTERNET__Si,FAMI_TIENEINTERNET__no info
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
692495,0.0,1.0,0.0
692496,0.0,1.0,0.0
692497,0.0,1.0,0.0
692498,1.0,0.0,0.0


In [49]:
datos = pd.concat([FAMI_TIENEINTERNET_df, datos], axis=1).drop('FAMI_TIENEINTERNET', axis=1)
datos.shape

(692500, 7)

In [50]:
datos.head()

,FAMI_TIENEINTERNET__No,FAMI_TIENEINTERNET__Si,FAMI_TIENEINTERNET__no info,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,RENDIMIENTO_GLOBAL
0,0.0,1.0,0.0,6.25,5,3,medio-alto
1,1.0,0.0,0.0,3.25,0,3,bajo
2,0.0,1.0,0.0,3.25,35,3,bajo
3,0.0,1.0,0.0,4.75,0,4,alto
4,0.0,1.0,0.0,3.25,25,3,medio-bajo


In [51]:
y = 'RENDIMIENTO_GLOBAL'

rg = { 'bajo':0, 'medio-bajo':1, 'medio-alto':2, 'alto': 3}
datos[y] = [rg[i] for i in datos[y]]

# Datos preprocesados

In [52]:
datos.head()

,FAMI_TIENEINTERNET__No,FAMI_TIENEINTERNET__Si,FAMI_TIENEINTERNET__no info,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,RENDIMIENTO_GLOBAL
0,0.0,1.0,0.0,6.25,5,3,2
1,1.0,0.0,0.0,3.25,0,3,0
2,0.0,1.0,0.0,3.25,35,3,0
3,0.0,1.0,0.0,4.75,0,4,3
4,0.0,1.0,0.0,3.25,25,3,1
